In [43]:
import pandas as pd
import numpy as np

import openai
from gensim.summarization import summarize
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import os

import sys

sys.path.insert(0, '..')

from src.features.youtubeCleaner import Cleaner

In [44]:
df = pd.read_csv('../data/interim/youtube/seed_videos_3.csv')

In [45]:
cleaner = Cleaner(df, save_path='../data/seed/youtube/notebook_seed_test.csv', use_ratio = True, ratio = 0.10)
df = cleaner.remove_nulls()

In [46]:
df.head()

,id,title,description,tags,cleaned_transcript,raw_transcript,comments,comment_ids,label,link
0,gFQNPmLKj1k,How to Invest for Beginners,"Hey friends, in this video I'll give you a com...","['Ali Abdaal', 'Ali abdal', 'how to get starte...","Right, so let's say\nyou want to get started w...","[{'text': ""Right, so let's say\nyou want to ge...","['@Ali Abdall , man i wantend to start from th...","['UgykIF0qeJ7ORZ-q8Dd4AaABAg', 'Ugz68Vae2si-Tu...",traditional,www.youtube.com/watch?v=gFQNPmLKj1k
1,GZ42PIi9bis,I asked a personal finance expert how to invest.,Go to https://squarespace.com/mattdavella to s...,"[""Matt D'Avella"", 'matt davella', 'self-develo...","- Investing, when most people think of it, the...","[{'text': '- Investing, when most people think...",['Huge thanks to Ramit Sethi for joining me! S...,"['Ugx7vuOGQxrEMVYfVyN4AaABAg', 'Ugx7vuOGQxrEMV...",traditional,www.youtube.com/watch?v=GZ42PIi9bis
2,Z3I8TPYn4Z0,Traditional and Roth IRAs | Simple Steps for a...,Individual Retirement Accounts (IRAs) can hel...,"['tdameritrade', 'TD Ameritrade', 'What is an ...","If you're saving for retirement, IRAs, also\nk...","[{'text': ""If you're saving for retirement, IR...",['hmmmmm do I think taxes will be HIGHER or LO...,"['UgxhIPwye-LL2ICHp594AaABAg', 'Ugw_poXl_sbpan...",traditional,www.youtube.com/watch?v=Z3I8TPYn4Z0
3,ss5yImerkhg,What is a 401(k)?,A 401(k) is an employer-sponsored retirement a...,"['401k', '401(k) contributions', '401k limit',...",One of the most common ways people save for\nr...,[{'text': 'One of the most common ways people ...,['so if the market crashes what happens to you...,"['UgxvZb1p0tmadP_Lrn94AaABAg', 'UgxvZb1p0tmadP...",traditional,www.youtube.com/watch?v=ss5yImerkhg
4,DPnXJpDOAHI,How IRAs Work And Why They Are More Popular Th...,Choosing the right retirement account that bes...,"['CNBC', 'business', 'news', 'finance stock', ...",are you one of the 60 million americans. acti...,[{'text': 'are you one of the 60 million ameri...,['I honestly hope they don’t pass laws that au...,"['UgyjWWVJM9zySO3L3oJ4AaABAg', 'Ugx-U2VntiMGdX...",traditional,www.youtube.com/watch?v=DPnXJpDOAHI


In [47]:
df_with_snippet = cleaner.generateVideoSnippets()

In [ ]:
def applyTfidf(corpus, n):
    """Apply TF-IDF"""
    # Should I just make one of these?
    # Convert the text into a sparse matrix using TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform([corpus])
    
    # Get the feature names and scores
    feature_names = vectorizer.get_feature_names_out()
    scores = dict(zip(feature_names, tfidf.data))
    
    # Sort the scores in descending order
    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)

    # Select only the top N features
    selected_features = [x[0] for x in sorted_scores[:n]]
    
    return selected_features

In [ ]:
def topTags(tags, n = 10):
    """Apply TF-IDF to the tags to get the ten most relevant tags"""
    corpus = " ".join([tag.strip().strip("\'") for tag in tags[1:-1].strip('[').strip(']').split(',')])
    
    if len(corpus) == 0:
        return ""
    
    selected_features = applyTfidf(corpus, n)
    
    return " ".join(selected_features)

In [ ]:
def condenseTranscript(transcript, summary = False, tfidf = False, word_count = 250, n = 50):
    """Cleans and condenses transcript"""
    corpus = transcript.replace("\n", " ").replace(" - ", "").replace('- ', "").replace("\'", "").replace(".", ". ")
    if summary:
        if len(corpus) < word_count:
            return corpus
        elif len(corpus.split(".")) <= 1:
            return corpus
        return summarize(corpus, word_count = word_count)
    elif tfidf:
        if len(corpus) < word_count:
            return corpus
        elif len(corpus.split(".")) <= 1:
            return corpus
        return " ".join(applyTfidf(corpus, n)) + "."
    else:
        return corpus

In [ ]:
def createVideoSnippet(title, transcript, tags):
    """Creates the video snippet"""
    return title + '. ' + transcript.replace("\n", " ") + ' ' + tags + "."

In [ ]:
def createPrompt(videos):
    """Create prompt given video snippet list"""
#     prompt="These labels determine if a video is discussing or recommending the following investments:\nAlternative: Cryptocurrency, Blockchain, NFTs\nTraditional: Stocks, Bonds, Real Estate, Commodities\nMixed: Discusses at least one of each topic from alternative and traditional labels defined above.\nNone: Not related to investing or finance\n\nClassify these YouTube video snippets, with each snippet containing the title, transcript, and video tags.\n\nFor each video return probabilities for all of the labels and explain the most probable label.\n\nExample Output:\nAlternative: 0.15 Traditional 0.15 Mixed .1 None .6 None because lorem ipsum."
#     prompt="These labels determine if a video is discussing or recommending the following investments:\nAlternative: Cryptocurrency, Blockchain, NFTs\nTraditional: Stocks, Bonds, Real Estate, Commodities\nMixed: Must at least one of each topic from alternative and traditional labels defined above.\nNone: Doesn't discuss the topics above or related topics.\n\nFor each video return probabilities for all of the labels and explain the most probable label.\nExample Output:\nAlternative: 0.15 Traditional 0.15 Mixed .1 None .6 None because lorem ipsum.\n\nClassify each YouTube video snippet below, with each snippet containing the title, transcript, and video tags."
    prompt="These labels determine if a video is discussing or recommending the following investments:\n1. Alternative: Cryptocurrency, Blockchain, NFTs\n2. Traditional: Stocks, Bonds, Real Estate, Commodities\n3. Mixed: Discusses at least one of each topic from alternative and traditional labels\n4. None: Not related to investing or finance\n\nClassify these YouTube video snippets, with each snippet containing the title, transcript, tags."
    for i in range(len(videos)):
        prompt += '\n\n{} '.format(i+1) + videos[i]
    prompt += "\n\nFor each video return probabilities for all of the labels and explain the most probable label.\nExample Output:\nAlternative: 0.15 Traditional: 0.15 Mixed: .1 None: .6 None because lorem ipsum"
    
    return prompt

In [ ]:
# Initialize the API client
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [ ]:
# Function to make classification requests
def classify(prompt, model_engine = "text-davinci-002", max_tokens = 1024, n = 1, temperature = 0.5):
    completions = openai.Completion.create(
        engine=model_engine,
        prompt=prompt,
        max_tokens=max_tokens,
        n=n,
        stop=None,
        temperature=temperature,
    )
    return completions.choices[0]

In [ ]:
df['snippet'] = df.apply(lambda x: createVideoSnippet(x.title, condenseTranscript(x.cleaned_transcript, summary = True), topTags(x.tags)), axis = 1)

In [ ]:
df['cleaned_transcript'][1]

In [ ]:
df['snippet'][1]

In [ ]:
prompt = createPrompt(videos)
print(prompt)

In [ ]:
prediction = classify(prompt)

In [ ]:
print(prediction.text)

In [ ]:
prompts = []
videos = []

for i in range(len(df)):
    title = df['title'][i]
    tags = df['tags'][i]
    transcript = df['cleaned_transcript'][i]
    
    if len(videos) >= 5:
        videos = []
        prompt = createPrompt(videos)
        prompts.append(prompt)
        
    videos.append(createVideoSnippet(title, topTags(tags), condenseTranscript(transcript, summary = True)))
    
prompt = createPrompt(videos)
prompts.append(prompt)

In [ ]:
print(prompts[0])

In [ ]:
predictions = []

for prompt in prompts:
    predictions.append(classify(prompt))

In [ ]:
print(predictions[0].text)